In [ ]:
# import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import csv

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from ecig_parsing import *


In [ ]:
def get_html(url, clicked=False, closed=False, elements=True):
    # print('GET', url)
    driver = None
    try:
#         # Set up headless mode for Firefox
        options = Options()
        options.add_argument('--headless')  # Enable headless mode properly

        # Start the WebDriver in headless mode
        driver = webdriver.Firefox(options=options)
#         driver = webdriver.Firefox()


        # Start the WebDriver in headless mode
        driver.get(url)
        last_n = 0
        hrefs = set()
        button_id = "yes-button"

        html = driver.page_source
        time.sleep(1)
        same_count = 0
        if not clicked:
            wait = WebDriverWait(driver, 15)  # 10 seconds timeout
            button = wait.until(EC.element_to_be_clickable((By.ID, button_id)))  # Use ID, XPATH, or other locator

            # Click the button
            button.click()
            clicked = True
#         if not closed:
#             wait = WebDriverWait(driver, 10)  # 10 seconds timeout
#             button = wait.until(EC.element_to_be_clickable((By.ID, 'omnisend-form-65006a9819ed2c118d9a9352-close-action')))  # Use ID, XPATH, or other locator

#             # Click the button
#             button.click()
#             closed = True
    

        # Set a delay for dynamic content loading
        SCROLL_PAUSE_TIME = 0.5  # Shorter pause for smoother scroll

        # Scroll incrementally by a small step (e.g., 500 pixels)
        scroll_increment = 500

        # Get the initial scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        first_height = last_height
        # print(last_height)

        while True:
            # Scroll down by the increment
            driver.execute_script(f"window.scrollBy(0, {scroll_increment});")

            # Wait for the page to load new content
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate the new scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # Check if new elements have appeared on the screen
            # You can check specific elements like this:
            # elements = driver.find_elements(By.CLASS_NAME, "your-element-class")

            # If the scroll height has not changed, break the loop
            #print(same_count, new_height, last_height)
            if new_height == last_height :
                same_count += 1
                if same_count >= 3:
                    break
            else:
                same_count = 0

            # Update the last height
            last_height = new_height
            
            html = driver.page_source
            #print(html)
        

    except Exception as ex:
        print(ex)
        html = ''
    finally:
        if driver:
            driver.close()




    return html


In [ ]:
import json
import os

# Specify the file path
json_file_path = 'vapingdotcom.json'

# Check if the file exists
if os.path.exists(json_file_path):
    # Open and read the JSON file
    with open(json_file_path, 'r') as file:
        try:
            found = json.load(file)
            print("JSON data:", len(found.keys()), 'items')
        except json.JSONDecodeError:
            print("Error: The file contains invalid JSON.")
else:
    found = dict()
    print(f"File '{json_file_path}' does not exist.")

In [ ]:


links = [
   'https://vaping.com/collections/all-products' 
]

In [ ]:
from collections import Counter
import csv
product_list = list()
all_headers = list()
header_counter = Counter(all_headers)
product_tags = set()
header_samples = dict()


def save():
    print('SAVING...')
    with open('vapingdotcom_scrape.csv', mode='w') as file:
        # Create a DictWriter object
        writer = csv.DictWriter(file, fieldnames=product_list[0].keys())

        # Write the header (column names)
        writer.writeheader()

        # Write the rows (each dictionary in the data list)
        writer.writerows(product_list)
        
    with open(json_file_path, 'w') as file:
        json.dump(found, file, indent=4) 
        
max_pages = 75
found_max = False
for link_index,l in enumerate(links):
    page = 1
    while True:
        if page > max_pages:
            break
        main_url = f'{l}?page={page}'
        try:
            if main_url in found:
                req_text = found[main_url]
            else:
                req_text = get_html(main_url)
                found[main_url] = req_text

            soup = BeautifulSoup(req_text)
            prods = soup.find_all('product-card')
            

        except Exception as ex:
            print(ex, 'FAILED', main_url)
            break
           
            
        total_items = len(prods)
        print('LINK INDEX', link_index, 'OF', len(links), ' -- PAGE', page, 'OF', max_pages, f'({total_items})', main_url)

            
        page += 1
        if total_items == 0:
            break
            
        for psoup in prods:
            txt = psoup.get_text().strip()
            if txt == '':
                continue
                
            product_name = psoup.find('a', class_='bold').text.strip()

            # Extract the product link
            product_link = psoup.find('a', class_='bold')['href']

            # Extract the sale price
            sale_price = psoup.find('sale-price').text.replace('Sale price', '').strip()

            # Extract the primary image URL and alt text
            primary_image_tag = psoup.find('img', class_='product-card__image--primary')
            if primary_image_tag:
                primary_image_url = primary_image_tag['src']
                primary_image_alt = primary_image_tag['alt']
            else:
                primary_image_url = None
                primary_image_alt = None

            # Extract the secondary image URL and alt text
            secondary_image_tag = psoup.find('img', class_='product-card__image--secondary')
            if secondary_image_tag:
                secondary_image_url = secondary_image_tag['src']
                secondary_image_alt = secondary_image_tag['alt']
            else:
                secondary_image_url = None
                secondary_image_alt = None
            
            if product_link.startswith('http'):
                full_link = product_link
            else:
                full_link = f'https://vaping.com{product_link}'
                
            if primary_image_url and primary_image_url.startswith('//'):
                primary_image_url = 'http:' + primary_image_url
                
            if secondary_image_url and secondary_image_url.startswith('//'):
                secondary_image_url = 'http:' + secondary_image_url
            
            tag = full_link.split('/')[-1]
            if tag == '':
                tag = full_link.split('/')[-2]
                
            print(tag, full_link)

            if tag in product_tags:
                continue
                
            try:
            
                if full_link in found:
                    reqtxt = found[full_link]
                else:
                    # print(full_link)
                    reqtxt = get_html(full_link, elements=False)
                    found[full_link] = reqtxt
            except Exception as ex:
                print(ex, 'FAILED', main_url)
                continue
                


                
            soup = BeautifulSoup(reqtxt)
            
            description = ''
            warning = ''
            package_contents = ''
            ingredients = ''
            key_features = ''
            flavors = ''
            specifications = ''
            performance = ''
            disposable = ''
            why = ''
            preference = ''
            innovation = ''
            enjoyment = ''
            compatibility = ''
            coil = ''
            battery = ''
            puffs = ''
            nicotine = ''
            e_liquid = ''
            prefilled = ''
            devices = ''
            airflow = ''
            heating_element = ''
            operation = ''
            charging = ''
            screen = ''
            links_txt = ''
            colors = ''
            options_txt = ''
            returns = ''
            size = ''
            pods = ''
            hardware = ''
            wattage = ''
            faqs = ''
            vgpg_ratio = ''
            power_mode = ''
            name = ''

            section_strings_raw = {
                'Digital Screen': 'screen',
                "FAQ's": "faqs",
                "FAQs": "faqs",
                "FAQ": "faqs",
                'LED Screen': 'screen',
                'Display Screen': 'screen',
                'Charging': 'charging',
                'Charging Port Info': 'charging',
                'Operation': 'operation',
                'Prefilled Capacity': 'prefilled',
                'Compatibility': 'compatibility',
                'Compatible with': 'compatibility',
                'Ingredients': 'ingredients',
                'INCLUDES': 'package_contents',
                'Kit Includes': 'package_contents',
                "What's Included": 'package_contents',
                'Package Content': 'package_contents',
                'Package Contents': 'package_contents',
                'Package Includes': 'package_contents',
                'Included in package': 'package_contents',
                'Included in the Package': 'package_contents',
                'Included': 'package_contents',
                'Includes': 'package_contents',
                'Contents': 'package_contents',
                "What’s Included": 'package_contents',
                'CALIFORNIA PROPOSITION 65 - Warning': 'warning',
                'CAUTION: this Product Contains Nicotine. Nicotine is an addictive chemical.': 'warning',
                'Warning': 'warning',
                'AVAILABLE OPTIONS': 'key_features',
                'Features and Specifications': 'key_features',
                'Key Features': 'key_features',
                'Features': 'key_features',
                'Flavors': 'flavors',
                'Flavor Selection': 'flavors',
                'Flavor Options': 'flavors',
                'Cutting-Edge Design and Technology:': 'key_features',
                'Superior Versatility and Performance:': 'key_features',
                'Flavor Selection:': 'flavors',
                'WARNING': 'warning',
                'Description': 'description',
                'Product Specifications': 'specifications',
                'Specifications': 'specifications',
                'Available Flavors': 'flavors',
                'Flavor Profiles': 'flavors',
                'Exceptional Flexibility and Performance': 'performance',
                'Disposable': 'disposable',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Tailored to Your Preference': 'preference',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Enjoy My Shisha on the Go': 'enjoyment',
                'The Ultimate Vaping Experience': 'enjoyment',
                'Elevate Your Vaping Experience': 'enjoyment',
                'Why Choose the Juice Head 30K Disposable?': 'why',
                'Why Geek Bar Skyview Disposable?': 'why',
                'Features and Specifications': 'key_features',
                'Hotbox Features': 'key_features',
                'Lost Mary x Elf Bar OS5000 Disposable Features': 'key_features',
                'Experience the Ultimate Vaping Innovation': 'innovation',
                'Flavor Details': 'flavors',
                'Advanced Dual Mesh Coil': 'coil',
                'Coil Specifications': 'coil',
                'Battery Info': 'battery',
                'Battery': 'battery',
                'Puffs per Device': 'puffs',
                'Puff Count': 'puffs',
                'Max Puffs': 'puffs',
                'What Is a Puff Count?': 'puffs',
                'Nicotine Strengths': 'nicotine',
                'Nicotine Level': 'nicotine',
                'Available Nicotine': 'nicotine',
                'Nicotine': 'nicotine',
                'Nicotine Strength': 'nicotine',
                'Battery Capacity': 'battery',
                'E-liquid Capacity': 'e_liquid',
                'E-liquid contents': 'e_liquid',
                'E-Liquid contents': 'e_liquid',
                'Maximum Puffs': 'puffs',
                'Available Flavor Profiles': 'flavors',
                'Prefilled': 'prefilled',
                'Compatible Devices': 'devices',
                'Devices': 'devices',
                'California Proposition 65 Warning': 'warning',
                'Melli Flavors': 'flavors',
                'Airflow': 'airflow',
                'Adjustable Airflow': 'airflow',
                'Heating Element': 'heating_element',
                'Quick Links': 'links',
                'Quick Link': 'links',
                'Related Links':'links',
                'Related Link':'links',
                'Colors': 'colors',
                'Color': 'colors',
                'Quick links': 'links',
                'Flavor': 'flavors',
                'Size': 'size',
                'Sizes': 'size',
                'Bottle Size': 'size',
                'Bottle Sizes': 'size',
                'Return and Exchange Policy Disclaimer:': 'returns',
                'Returns': 'returns',
                'Pods': 'pods',
                'Pod Info': 'pods',
                'Options': 'options',
                'Option': 'options',
                'Compatible Hardware': 'hardware',
                'Hardware': 'hardware',
                'Wattage Info': 'wattage',
                'Wattage': 'wattage',
                'Coil': 'coil',
                'Coils': 'coil',
                'Available Nicotine': 'nicotine',
                'How Are Disposables Different Than Others?': 'disposable',
                'VG/PG Ratio': 'vgpg_ratio',
                'VGPG Ratio': 'vgpg_ratio',
                'VG-PG Ratio': 'vgpg_ratio',
                'Vg/Pg Ratio': 'vgpg_ratio',
                'Power Mode Settings': 'power_mode',
                'Power Mode': 'power_mode',
                '3D Curved Screen': 'screen',
                'Name Change': 'name',
                'Variable Wattage': 'wattage',
                'Disposable Vape Battery': 'battery',
                'Battery Pack/Charging Dock': 'battery'

            }
            
            functions = [
                str.upper,
                str.lower,
                str.capitalize,
                str.title,
                str.swapcase,
                str.casefold
            ]
            section_strings = dict()
            for k, v in section_strings_raw.items():
                key = k
                section_strings[key] = v
                if key.endswith(':'):
                    key = key.replace(':', '')
                    section_strings[key] = v
                    
                    key = key.strip() + '-'
                    section_strings[key] = v
                    key = key.strip() + '--'
                    section_strings[key] = v
                    key = key.strip() + ' - '
                    section_strings[key] = v
                else:
                    key = key.strip() + ':'
                    section_strings[key] = v
                    
                    key = key.strip() + '-'
                    section_strings[key] = v
                    key = key.strip() + '--'
                    section_strings[key] = v
                    key = key.strip() + ' - '
                    section_strings[key] = v
                    
                key = key.strip() + '::'
                section_strings[key] = v
                
                for func in functions:
                    key = func(k)
                    section_strings[key] = v
                    if key.endswith(':'):
                        key = key.replace(':', '')
                        section_strings[key] = v

                        key = key.strip() + '-'
                        section_strings[key] = v
                        key = key.strip() + '--'
                        section_strings[key] = v
                        key = key.strip() + ' - '
                        section_strings[key] = v
                    else:
                        key = key.strip() + ':'
                        section_strings[key] = v

                        key = key.strip() + '-'
                        section_strings[key] = v
                        key = key.strip() + '--'
                        section_strings[key] = v
                        key = key.strip() + ' - '
                        section_strings[key] = v

                    key = key.strip() + '::'
                    section_strings[key] = v
                
            
            section_map = {
                'description': description,
                'warning': warning,
                'package_contents': package_contents,
                'ingredients': ingredients,
                'key_features': key_features,
                'flavors': flavors,
                'specifications': specifications,
                'performance': performance,
                'disposable': disposable,
                'why': why,
                'innovation': innovation,
                'preference': preference,
                'enjoyment': enjoyment,
                'compatibility': compatibility,
                'coil': coil,
                'battery': battery,
                'puffs': puffs,
                'nicotine': nicotine,
                'e_liquid': e_liquid,
                'prefilled': prefilled,
                'devices': devices,
                'airflow': airflow,
                'heating_element': heating_element,
                'operation': operation,
                'charging': charging,
                'screen': screen,
                'links': links_txt,
                'colors': colors,
                'returns': returns,
                'options': options_txt,
                'size': size,
                'pods': pods,
                'hardware': hardware,
                'wattage': wattage,
                'faqs': faqs,
                'vgpg_ratio': vgpg_ratio,
                'power_mode': power_mode,
                'name': name
            }

            desc_section = soup.find('div', {'class': 'product-info__description'})
            if desc_section:
                cur_section = 'description'
                if desc_section:
                    headers = [tag.text for tag in desc_section.find_all(lambda tag: tag.name in ['strong','h1', 'h2', 'h3', 'h4', 'h5'])]
                    stext = desc_section.get_text(separator='\n').split('\n')
                    for s in stext:
                        s = s.strip()
                        if s == '':
                            continue
                        s_no_colon = s.replace(':', '').strip()
                        if s in section_strings:
                            cur_section = section_strings[s]
                        elif s_no_colon in section_strings:
                            cur_section = section_strings[s_no_colon]
                        elif s.lower().startswith('why') and s.endswith('?'):
                            cur_section = 'why'
                        elif 'features:' in s.lower():
                            cur_section = 'key_features'
                        elif s in headers:
                            all_headers.append(s)
                            header_samples[s] = full_link
                            header_counter = Counter(all_headers)
                            #if header_counter[s] > 5:
                            #    print(s, full_link)
                            cur_section = 'description'
                            section_map[cur_section] += f'{s}\n\n'
                        else:
                            section_map[cur_section] += f'{s}\n'
                            
            fieldsets = soup.find_all('fieldset', {'class': 'variant-picker__option'})
            size_values = []
            nicotine_values = []
            color_values = []
            for fs in fieldsets:
                txt = fs.get_text().strip()
                if 'Nicotine:' in txt:
                    radio_inputs = fs.find_all('input', {'type': 'radio'})

                    # Extract values from the input elements
                    nicotine_values = [input_tag['value'] for input_tag in radio_inputs]
                elif 'Size:' in txt:
                    radio_inputs = fs.find_all('input', {'type': 'radio'})

                    # Extract values from the input elements
                    size_values = [input_tag['value'] for input_tag in radio_inputs]
                elif 'Color:' in txt:
                    radio_inputs = fs.find_all('input', {'type': 'radio'})

                    # Extract values from the input elements
                    color_values = [input_tag['value'] for input_tag in radio_inputs]
                
                
            brand_div = soup.find('div', {'class': 'product-info__vendor'})
            if brand_div:
                brand_name = brand_div.get_text().strip()
            else:
                brand_name = ''
                
            stock_div = soup.find('div', {'class': 'product-info__inventory'})
            if stock_div:
                stock_status = stock_div.get_text().strip()
            else:
                stock_status = ''
                
            warning_div = soup.find('div', {'class': 'prop-65-text'})
            if warning_div:
                warning = warning_div.get_text().strip()
            else:
                warning = ''
                
            full_image_urls = []
            thumbnails = soup.find('scroll-shadow', {'class': 'product-gallery__thumbnail-list-wrapper'})
            if thumbnails:
                # Find all img tags and extract src attribute
                image_urls = [img['src'] for img in thumbnails.find_all('img')]

                # Prefix the URLs with "https:" to make them fully accessible
                for u in image_urls:
                    if 'svg' in u:
                        continue
                    if u.startswith('//'):
                        u = "https:" + u
                    full_image_urls.append(u)
            
            product_data = {
                'tag': tag,
                "title": product_name,
                'brand': brand_name,
                'stock_status': stock_status,
                "link": full_link,
                "price": sale_price,
                'image_url': primary_image_url,
                'image_alt_text': primary_image_alt,
                'image2_url': secondary_image_url,
                'image2_alt_text': secondary_image_alt,
                'image_urls': full_image_urls,
                'sizes': size_values,
                'nicotine_values': nicotine_values,
                'colors': color_values,
                'warning': warning,

            }
            
            if primary_image_url:
                download_image(primary_image_url, f'{tag}', save_dir='vapingdotcom_images')
                
            if secondary_image_url:
                download_image(secondary_image_url, f'{tag}_secondary', save_dir='vapingdotcom_images')

            if full_image_urls:
                image_num = 0
                for i in product_data['image_urls']:
                    if i and i != '':
                        download_image(i, f'{tag}-image{image_num}', save_dir='vapingdotcom_images')
                        image_num += 1
                    
            desc_fields = ''
            for s, v in section_map.items():
                if 'description' not in s:
                    s = s + '_description'
                val = v.replace('\xa0', ' ').strip()
                
                if val.startswith(':'):
                    val = val[1:].strip()
            
                product_data[s] = val
                if product_data[s] != '' and 'link' not in s:
                    desc_fields += f'\n{product_data[s]}'

            feats = list()
            feat = find_features(desc_fields)
            any_found, puffs_res, nico_res, ml_res, flav_text, dev_text = feat

            if any_found:
                #print(feat)
                feats.append(feat)
            disposable,recharge,battery,mesh,usb,adjustable,found_flavs = features_to_cats(feats)
            product_data['disposable'] = disposable
            product_data['rechargeable'] = recharge
            product_data['battery'] = battery
            product_data['mesh'] = mesh
            product_data['usb'] = usb
            product_data['adjustable'] = adjustable

            product_list.append(product_data)
            product_tags.add(tag)
        
            if len(product_list) % 50 == 0:
                rc = (random.choice(product_list))
                print('{')
                for key, value in rc.items():
                    if not value:
                        continue

                    if isinstance(value, list):
                        print(f"\t{key}: {len(value)} items")
                    else:
                        value = value.replace('\n', ' ')

                        if key == 'image_url' or key == 'link' or key == 'title'  or key == 'tag' :
                            print(f"\t{key}: {value}")
                        elif 1 <= len(value) <= 20:
                            print(f"\t{key}: {value}")
                        elif len(value) > 20:
                            print(f"\t{key}: {value[:20]}...")
                print('}')
        
                print(len(product_list), 'ITEMS FOUND')
                print()
                print()
                for item, count in header_counter.most_common(50):
                    if count > 5:
                        print(f"{item}: {count} - {header_samples[item]}")
                print()
                print()


            
        save()
    save()
save()
        
random.choice(product_list)